# Deep learning from scratch: homework 4

### General instructions

Complete the exericses listed below in this Jupyter notebook - leaving all of your code in Python cells in the notebook itself.

### When submitting this homework:

**Make sure you have put your name at the top of this file**

**This is the only file you must submit** 
    
**Make sure all output is present in your notebook prior to submission**

**Do not zip your files when uploading to canvas**

In [2]:
# imports necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec
from PIL import Image

# this is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

-----------

#### <span style="color:#a50e3e;">Exercise 1. </span> ZCA sphereing, K-means clustering, and natural image patches

In this Exercise you will use the K-means algorithm (see [Section 11.5 of the course notes](https://jermwatt.github.io/mlrefined/blog_posts/11_Linear_unsupervised_learning/11_5_K_means_clustering.html)) to cluster a set of natural image patches.  

First, you need to complete the `extract_patches` module below. This module will extract a total of `number_of_pathes` randomly-positioned patches of size `patch_size` from megapixel images listed in `image_list`. When coding `extract_patches` make sure to exclude flat or nearly flat patches, i.e., those wherein the standard deviation of pixel values falls below $0.1$.

In [3]:
# Hint: you can use the line below to convert an image to a numpy array  
# image_as_numpy_array = np.array(Image.open(path_to_image).convert('L'))

In [1]:
def extract_patches(image_list, number_of_patches, patch_size):

    # YOUR CODE GOES HERE.

    return patches

Once you have coded up `extract_patches`, activate the following cell to extract $100000$ image patches, each of size $12 \times 12$, from the four images given below.

In [ ]:
image_1 = 'images/bean.jpg'
image_2 = 'images/dog.jpg'
image_3 = 'images/flyer.jpg'
image_4 = 'images/Trey_Matt.png'

image_list = [image_1, image_2, image_3, image_4]
number_of_patches = 100000
patch_size = 12
              
patches = extract_patches(image_list, number_of_patches, patch_size)   

Notice, `patches` must be a matrix of size $144 \times 100000$ where each column is a flattened/vectorized $12 \times 12$ patch. The cell below plots the first $100$ patches in `patches` (Note: each column has to be reshaped back into a square before plotting).

In [ ]:
def show_images(X):
    '''
    Function for plotting input images, stacked in columns of input X.
    '''
    # plotting mechanism taken from excellent answer from stack overflow: https://stackoverflow.com/questions/20057260/how-to-remove-gaps-between-subplots-in-matplotlib
    plt.figure(figsize = (6,6))
    gs1 = gridspec.GridSpec(10, 10)
    gs1.update(wspace=0.05, hspace=0.05) # set the spacing between axes. 
    
    # shape of square version of image
    square_shape = int((X.shape[0])**(0.5))

    for i in range(min(100,X.shape[1])):
        # plot image in panel
        ax = plt.subplot(gs1[i])
        im = ax.imshow(np.reshape(X[:,i],(square_shape,square_shape)),cmap = 'gray')

        # clean up panel
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])

    plt.show()
    
# Plot the first 100 patches 
show_images(patches)

<figure><img src="Figures/sample_patches.png" width="50%" height="auto"></figure>

Now perform ZCA sphereing to normalize `patches`. To do this, you may use and adjust the PCA sphereing code provided in [Section 11.3 of the course notes](https://jermwatt.github.io/mlrefined/blog_posts/11_Linear_unsupervised_learning/11_3_PCA_sphereing.html). Remember, ZCA sphereing differs from PCA sphereing by one simple step (See Figure below and compare with Figure 1 in [Section 11.3 of the course notes](https://jermwatt.github.io/mlrefined/blog_posts/11_Linear_unsupervised_learning/11_3_PCA_sphereing.html)).    

<figure><img src="Figures/ZCA_diagram.png"></figure>

In [1]:
# YOUR CODE GOES HERE
# patches_ZCA_normalized = ...

Activate the cell below to plot the first $100$ patches in `patches_ZCA_normalized` - this is the matrix containing ZCA-normalized patches.

In [ ]:
show_images(patches_ZCA_normalized)

<figure><img src="Figures/sample_ZCA.png" width="50%" height="auto"></figure>

To perform clustering on ZCA normalized patches we make use of `scikit-learn`'s efficient implementation of the K-means algorithm. Activating the cell below will run K-means for a maximum of $2000$ iterations with learned centroids stored in the matrix `centroids`.       

In [ ]:
# perform K-means clustering
from sklearn.cluster import KMeans

# number of clusters
num_clusters = 100 

clusterer = KMeans(n_clusters=num_clusters, max_iter = 2000, n_init = 1)

# fit the algorithm to our dataset
clusterer.fit(patches_ZCA_normalized.T)

# extract cluster centroids
centroids = clusterer.cluster_centers_.T

Each centroid iself can be viewed as a patch when reshaped into a $12 \times 12$ matrix. Python cell below plots all these centroids.   

In [ ]:
show_images(centroids)

<figure><img src="Figures/sample_centroids.png" width="50%" height="auto"></figure>

A majority of these learned centroids look like edge-detectors, e.g., the centroid highlighted in the Figure below representing a horizontal edge detector. 

<figure><img src="Figures/edge_detector.png" width="70%" height="auto"></figure>

Can you explain why learned centroids resemble edge detectors of varying width and orientation? Hint: see [Section 14.2.1 of the course notes](https://jermwatt.github.io/mlrefined/blog_posts/14_Convolutional_networks/14_2_Edge_histogram_based_features.html).    

##### YOUR ANSWER GOES HERE

Finally, run K-means again, this time on the data stored in `patches` and plot the learned centroids.       

In [2]:
# YOUR CODE GOES HERE

<figure><img src="Figures/sample_unnormalized.png" width="50%" height="auto"></figure>

As you can see, without normalization, K-means algorithm is incapable of learning edge detector-like patches from the data.  

------------

#### <span style="color:#a50e3e;">Exercise 2. </span> Counting the number of tunable weights in a simple CNN 

Suppose you want to perform two-class cassification using convolutional neural networks on a training dataset consisting of $50,000$ face and non-face images, each of size $32 \times 32$. Your CNN has a single convolutional layer followed by a three (hidden) layer MLP as shown in the Figure below.      

<figure><img src="Figures/pipe_1.png"></figure>

In this Exercise you will find the total number of weights to be learned using this CNN architecture having the following specifications:

- Number of convolutional kernels: $10$
- Size of each convolutional kernel: $3\times 3$
- Pooling window size: $4\times 4$
- Pooling stride = 2 
- Number of units/neurons per layer of MLP: 5

##### YOUR ANSWER GOES HERE